In [1]:
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range


C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Parameters Config

In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'


In [3]:
TRAINING_SIZE = 30000
DIGITS = 3

REVERSE = True

MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789+- '

RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1


In [4]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)


In [5]:
ctable = CharacterTable(chars)


# Data Generation

In [6]:
questions = []
expected = []
seen = set()

print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                            for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    
    # if operator == 1 do adder, if operator == 2 do substractor
    operator = np.random.choice(list('12'))
    if operator == '1' :
        
        # Pad the data with spaces such that it is always MAXLEN.
        q = '{}+{}'.format(a, b)
        query = q + ' ' * (MAXLEN - len(q))
        
        ans = str(a + b)
        # Answers can be of maximum size DIGITS + 1.
        ans += ' ' * (DIGITS + 1 - len(ans))
        
    elif operator == '2' :
        # Pad the data with spaces such that it is always MAXLEN.
        q = '{}-{}'.format(a, b)
        query = q + ' ' * (MAXLEN - len(q))
        
        ans = str(a - b)
        # Answers can be of maximum size DIGITS + 1.
        ans += ' ' * (DIGITS + 1 - len(ans))

    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))



Generating data...
Total addition questions: 30000


In [7]:
print(questions[:10], '\n' ,expected[:10])


['91+27  ', '33+12  ', '5-93   ', '8+2    ', '419-683', '711-2  ', '99+583 ', '938-93 ', '8+1    ', '570-3  '] 
 ['118 ', '45  ', '-88 ', '10  ', '-264', '709 ', '682 ', '845 ', '9   ', '567 ']


# Processing

In [8]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)


Vectorization...


In [9]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:25000]
train_y = y[:25000]
test_x = x[25000:]
test_y = y[25000:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)



Training Data:
(22500, 7, 13)
(22500, 4, 13)
Validation Data:
(2500, 7, 13)
(2500, 4, 13)
Testing Data:
(5000, 7, 13)
(5000, 4, 13)


In [10]:
#print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])


# Build Model

In [11]:
print('Build model...')

############################################
##### Build your own model here ############
model = Sequential()

# "Encode" the input sequence using an RNN, input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))

# the decoder RNN's input
model.add(layers.RepeatVector(DIGITS + 1))

# The decoder RNN  with a single layer.
for _ in range(LAYERS):
    # all the outputs in the form of (num_samples, timesteps, output_dim). 
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))


model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))

############################################
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()


Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72704     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 13)             1677      
Total params: 205,965
Trainable params: 205,965
Non-trainable params: 0
_________________________________________________________________


# Training

In [12]:
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    # visualizing validation set prediction errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)



--------------------------------------------------
Iteration 0
Train on 22500 samples, validate on 2500 samples
Epoch 1/1
22500/22500 [==============================] - 9s 385us/step - loss: 2.0607 - acc: 0.2860 - val_loss: 1.8176 - val_acc: 0.3422
Q  446-97 T -565 ☒ -229
Q  279-75 T -915 ☒ -229
Q  193-26 T -329 ☒ -226
Q  401+09 T 194  ☒ 119 
Q 863+433 T 702  ☒ 112 
Q  02-992 T 279  ☒ -22 
Q 719+694 T 1413 ☒ 119 
Q  22-409 T 882  ☒ 122 
Q  188+99 T 980  ☒ 119 
Q  64+801 T 154  ☒ 112 

--------------------------------------------------
Iteration 1
Train on 22500 samples, validate on 2500 samples
Epoch 1/1
22500/22500 [==============================] - 6s 272us/step - loss: 1.7858 - acc: 0.3519 - val_loss: 1.7549 - val_acc: 0.3628
Q   753+4 T 361  ☒ 13  
Q   2+555 T 557  ☒ 13  
Q 673+457 T 1130 ☒ 105 
Q  495-24 T -552 ☒ -410
Q   6+731 T 143  ☒ 10  
Q  99+216 T 711  ☒ 105 
Q 906-729 T 318  ☒ -25 
Q  55-533 T 280  ☒ -3  
Q 826-454 T -174 ☒ -15 
Q   5+471 T 179  ☒ 13  

-------------------

22500/22500 [==============================] - 7s 305us/step - loss: 1.2472 - acc: 0.5392 - val_loss: 1.2535 - val_acc: 0.5309
Q  613+24 T 358  ☒ 494 
Q  833+47 T 412  ☒ 422 
Q 826-927 T 101  ☒ 21  
Q  68-717 T 631  ☒ 706 
Q   1-479 T 973  ☒ 990 
Q   921-2 T -127 ☒ -190
Q  32-681 T 163  ☒ 160 
Q  16+475 T 635  ☒ 614 
Q 806+538 T 1443 ☒ 1647
Q  22-787 T 765  ☒ 756 

--------------------------------------------------
Iteration 15
Train on 22500 samples, validate on 2500 samples
Epoch 1/1
22500/22500 [==============================] - 7s 314us/step - loss: 1.2156 - acc: 0.5494 - val_loss: 1.2191 - val_acc: 0.5486
Q   0-197 T 791  ☒ 785 
Q 967+066 T 1429 ☒ 1436
Q 442+381 T 427  ☒ 347 
Q   591+9 T 204  ☒ 185 
Q 128+038 T 1651 ☒ 1674
Q  16-072 T 209  ☒ 240 
Q  14+284 T 523  ☒ 520 
Q 362-089 T 717  ☒ 845 
Q    26+7 T 69   ☒ 63  
Q  82-755 T 529  ☒ 525 

--------------------------------------------------
Iteration 16
Train on 22500 samples, validate on 2500 samples
Epoch 1/1
22500/22500 [=====

22500/22500 [==============================] - 10s 426us/step - loss: 0.7971 - acc: 0.7100 - val_loss: 0.9082 - val_acc: 0.6637
Q 395+895 T 1191 ☒ 1198
Q   5-974 T 474  ☒ 472 
Q 282+145 T 823  ☒ 719 
Q 196-383 T -308 ☒ -288
Q  85+075 T 628  ☒ 636 
Q  919+33 T 952  ☒ 941 
Q   3+498 T 897  ☒ 896 
Q   913+5 T 324  ☒ 319 
Q   5+485 T 589  ☒ 582 
Q   168+3 T 864  ☑ 864 

--------------------------------------------------
Iteration 43
Train on 22500 samples, validate on 2500 samples
Epoch 1/1
22500/22500 [==============================] - 9s 410us/step - loss: 0.7854 - acc: 0.7134 - val_loss: 0.9008 - val_acc: 0.6670
Q  913+61 T 335  ☒ 347 
Q   62-44 T 18   ☒ 21  
Q  03+212 T 242  ☒ 244 
Q  41-728 T 813  ☒ 818 
Q  011+39 T 203  ☒ 294 
Q   4+654 T 460  ☒ 462 
Q   52+57 T 100  ☒ 90  
Q  097-58 T -705 ☒ -702
Q   937-6 T -733 ☒ -731
Q  13-107 T 670  ☒ 660 

--------------------------------------------------
Iteration 44
Train on 22500 samples, validate on 2500 samples
Epoch 1/1
22500/22500 [====

22500/22500 [==============================] - 12s 514us/step - loss: 0.3728 - acc: 0.8759 - val_loss: 0.5765 - val_acc: 0.7807
Q   7-031 T 123  ☒ 124 
Q  479-59 T -879 ☒ -878
Q   782+2 T 289  ☑ 289 
Q   3+531 T 138  ☑ 138 
Q 492+513 T 609  ☒ 618 
Q   59-28 T -13  ☒ -17 
Q   58+09 T 175  ☑ 175 
Q   938+5 T 844  ☑ 844 
Q   008+2 T 802  ☒ 803 
Q  14-109 T 860  ☑ 860 

--------------------------------------------------
Iteration 71
Train on 22500 samples, validate on 2500 samples
Epoch 1/1
22500/22500 [==============================] - 12s 533us/step - loss: 0.3596 - acc: 0.8809 - val_loss: 0.5742 - val_acc: 0.7762
Q 287-511 T -667 ☒ -620
Q   2+643 T 348  ☒ 347 
Q 557-464 T -291 ☒ -398
Q  08+602 T 286  ☑ 286 
Q    64+3 T 49   ☒ 40  
Q  44+555 T 599  ☒ 698 
Q   1-334 T 432  ☑ 432 
Q  402+76 T 271  ☒ 272 
Q  173-75 T -314 ☑ -314
Q   04+04 T 80   ☑ 80  

--------------------------------------------------
Iteration 72
Train on 22500 samples, validate on 2500 samples
Epoch 1/1
22500/22500 [===

# Testing

In [13]:
print("MSG : Prediction")
#####################################################
## Try to test and evaluate your model ##############
## ex. test_x = ["555+175", "860+7  ", "340+29 "]
## ex. test_y = ["730 ", "867 ", "369 "] 

preds = model.predict_classes(test_x, verbose=0)

i = 0
count = 0
while i < len(preds) :
    q = ctable.decode(test_x[i])
    correct = ctable.decode(test_y[i])
    guess = ctable.decode(preds[i], calc_argmax=False)

    print('Q', q, end=' ')
    print('T', correct, end=' ')
    if correct == guess:
        print(colors.ok + '☑' + colors.close, end=' ')
        count += 1
    else:
        print(colors.fail + '☒' + colors.close, end=' ')
    print(guess)
    i += 1
    
testing_acc = (count/len(test_y))*100
print('-'*60)
print('-'*30, colors.ok + str(testing_acc) + "%" + colors.close, '-'*30)
print('-'*60)
#####################################################
    

MSG : Prediction
Q 939+957 T 1896 ☒ 1807
Q 376+38  T 414  ☑ 414 
Q 23-887  T -864 ☑ -864
Q 568+876 T 1444 ☑ 1444
Q 675-971 T -296 ☒ -306
Q 526-991 T -465 ☒ -467
Q 52+35   T 87   ☑ 87  
Q 96+828  T 924  ☒ 914 
Q 55-164  T -109 ☒ -110
Q 389+56  T 445  ☒ 444 
Q 246+40  T 286  ☒ 386 
Q 135+210 T 345  ☒ 335 
Q 706+691 T 1397 ☒ 1488
Q 64+76   T 140  ☑ 140 
Q 859-4   T 855  ☑ 855 
Q 896+11  T 907  ☒ 908 
Q 62+18   T 80   ☑ 80  
Q 3+491   T 494  ☒ 484 
Q 69-183  T -114 ☒ -125
Q 88-65   T 23   ☑ 23  
Q 613-774 T -161 ☒ -170
Q 719-1   T 718  ☑ 718 
Q 830-36  T 794  ☑ 794 
Q 820+53  T 873  ☒ 874 
Q 892-750 T 142  ☒ 118 
Q 904-743 T 161  ☒ 43  
Q 533+35  T 568  ☒ 578 
Q 59+642  T 701  ☑ 701 
Q 754+1   T 755  ☑ 755 
Q 367-851 T -484 ☒ -485
Q 780-33  T 747  ☑ 747 
Q 53+121  T 174  ☒ 164 
Q 107-29  T 78   ☒ 11  
Q 847-2   T 845  ☑ 845 
Q 413-7   T 406  ☑ 406 
Q 464+7   T 471  ☑ 471 
Q 492-61  T 431  ☒ 420 
Q 98+198  T 296  ☑ 296 
Q 9-593   T -584 ☑ -584
Q 967+8   T 975  ☑ 975 
Q 335-3   T 332  ☑ 332 

Q 53-630  T -577 ☒ -578
Q 934-631 T 303  ☒ 204 
Q 893+75  T 968  ☑ 968 
Q 262+12  T 274  ☑ 274 
Q 828+604 T 1432 ☒ 1422
Q 483-539 T -56  ☒ -8  
Q 811-439 T 372  ☑ 372 
Q 913+842 T 1755 ☒ 1745
Q 45-1    T 44   ☒ 45  
Q 6+677   T 683  ☑ 683 
Q 789-72  T 717  ☒ 707 
Q 62-793  T -731 ☑ -731
Q 55-226  T -171 ☒ -181
Q 20+95   T 115  ☒ 116 
Q 82+19   T 101  ☑ 101 
Q 72-20   T 52   ☒ 51  
Q 936-2   T 934  ☑ 934 
Q 6-126   T -120 ☑ -120
Q 876+669 T 1545 ☒ 1635
Q 462+2   T 464  ☑ 464 
Q 233-889 T -656 ☑ -656
Q 730+21  T 751  ☑ 751 
Q 493+63  T 556  ☑ 556 
Q 552-872 T -320 ☑ -320
Q 412+156 T 568  ☒ 578 
Q 941-6   T 935  ☑ 935 
Q 56-784  T -728 ☑ -728
Q 90+888  T 978  ☑ 978 
Q 490+106 T 596  ☒ 697 
Q 24-955  T -931 ☑ -931
Q 21+0    T 21   ☒ 22  
Q 22-353  T -331 ☒ -320
Q 45-37   T 8    ☒ 1   
Q 528+631 T 1159 ☒ 1160
Q 422+55  T 477  ☑ 477 
Q 740-8   T 732  ☑ 732 
Q 378+625 T 1003 ☒ 9032
Q 589+613 T 1202 ☒ 1201
Q 362+139 T 501  ☒ 481 
Q 84+308  T 392  ☒ 301 
Q 3-878   T -875 ☑ -875
Q 676+3   T 679 

Q 209+1   T 210  ☒ 200 
Q 59-213  T -154 ☑ -154
Q 5-513   T -508 ☑ -508
Q 68+785  T 853  ☑ 853 
Q 25+13   T 38   ☑ 38  
Q 100+9   T 109  ☒ 100 
Q 302-5   T 297  ☑ 297 
Q 152+43  T 195  ☒ 295 
Q 96-775  T -679 ☒ -671
Q 5+429   T 434  ☑ 434 
Q 469+704 T 1173 ☒ 1162
Q 90+552  T 642  ☑ 642 
Q 898+903 T 1801 ☒ 1883
Q 52+12   T 64   ☑ 64  
Q 970-1   T 969  ☒ 979 
Q 0+726   T 726  ☑ 726 
Q 67-66   T 1    ☒ 0   
Q 254+275 T 529  ☑ 529 
Q 44-504  T -460 ☒ -461
Q 6+393   T 399  ☒ 398 
Q 735+12  T 747  ☑ 747 
Q 870-569 T 301  ☒ 211 
Q 499-10  T 489  ☑ 489 
Q 847+85  T 932  ☒ 931 
Q 4+986   T 990  ☑ 990 
Q 9-550   T -541 ☒ -542
Q 179+8   T 187  ☑ 187 
Q 8+176   T 184  ☑ 184 
Q 592+29  T 621  ☒ 622 
Q 927+3   T 930  ☒ 920 
Q 283+62  T 345  ☑ 345 
Q 123-673 T -550 ☒ -530
Q 266-94  T 172  ☒ 182 
Q 914+46  T 960  ☒ 950 
Q 20-26   T -6   ☒ -5  
Q 73-89   T -16  ☒ -17 
Q 381+922 T 1303 ☒ 1302
Q 320-64  T 256  ☑ 256 
Q 24-533  T -509 ☒ -510
Q 562+15  T 577  ☒ 578 
Q 35-995  T -960 ☑ -960
Q 911+483 T 1394

Q 898+536 T 1434 ☒ 1444
Q 5+295   T 300  ☒ 290 
Q 516-463 T 53   ☒ 1   
Q 65+65   T 130  ☑ 130 
Q 470-30  T 440  ☒ 430 
Q 61+418  T 479  ☑ 479 
Q 284-762 T -478 ☑ -478
Q 58-58   T 0    ☑ 0   
Q 83-928  T -845 ☒ -846
Q 93-682  T -589 ☒ -590
Q 514-70  T 444  ☒ 435 
Q 25+73   T 98   ☑ 98  
Q 94+357  T 451  ☒ 441 
Q 1+253   T 254  ☑ 254 
Q 578-84  T 494  ☒ 504 
Q 5-608   T -603 ☑ -603
Q 80+334  T 414  ☒ 424 
Q 57+13   T 70   ☒ 69  
Q 903-84  T 819  ☑ 819 
Q 491-8   T 483  ☑ 483 
Q 215-36  T 179  ☒ 170 
Q 25-54   T -29  ☑ -29 
Q 46+40   T 86   ☑ 86  
Q 165+74  T 239  ☑ 239 
Q 665+396 T 1061 ☒ 1041
Q 2-11    T -9   ☒ -10 
Q 88+15   T 103  ☑ 103 
Q 705-962 T -257 ☒ -277
Q 54+77   T 131  ☑ 131 
Q 81+32   T 113  ☑ 113 
Q 556+51  T 607  ☒ 698 
Q 243-949 T -706 ☑ -706
Q 42-741  T -699 ☒ -799
Q 20+89   T 109  ☑ 109 
Q 40-924  T -884 ☑ -884
Q 9-231   T -222 ☑ -222
Q 209+90  T 299  ☒ 209 
Q 604+36  T 640  ☒ 649 
Q 62-964  T -902 ☒ -802
Q 502+43  T 545  ☑ 545 
Q 36-54   T -18  ☑ -18 
Q 158-80  T 78  

Q 506-99  T 407  ☑ 407 
Q 842+18  T 860  ☒ 850 
Q 255-91  T 164  ☑ 164 
Q 64+149  T 213  ☑ 213 
Q 707-7   T 700  ☒ 790 
Q 760+1   T 761  ☑ 761 
Q 82+336  T 418  ☒ 428 
Q 2+355   T 357  ☑ 357 
Q 165+869 T 1034 ☑ 1034
Q 810-38  T 772  ☑ 772 
Q 255-83  T 172  ☒ 182 
Q 0-252   T -252 ☒ -251
Q 360-615 T -255 ☒ -276
Q 8+671   T 679  ☑ 679 
Q 44-458  T -414 ☑ -414
Q 4-724   T -720 ☒ -710
Q 114+64  T 178  ☑ 178 
Q 686-683 T 3    ☒ -   
Q 949+55  T 1004 ☑ 1004
Q 503+4   T 507  ☑ 507 
Q 972+1   T 973  ☑ 973 
Q 6-627   T -621 ☑ -621
Q 7-98    T -91  ☒ -90 
Q 75+61   T 136  ☑ 136 
Q 30-374  T -344 ☑ -344
Q 767-13  T 754  ☑ 754 
Q 0-844   T -844 ☒ -843
Q 22-917  T -895 ☒ -905
Q 9-253   T -244 ☑ -244
Q 144+67  T 211  ☒ 201 
Q 69-665  T -596 ☒ -696
Q 734+9   T 743  ☑ 743 
Q 44-44   T 0    ☑ 0   
Q 5-652   T -647 ☑ -647
Q 62-89   T -27  ☑ -27 
Q 773-5   T 768  ☒ 767 
Q 5+590   T 595  ☑ 595 
Q 49-87   T -38  ☑ -38 
Q 184+50  T 234  ☑ 234 
Q 50-399  T -349 ☒ -340
Q 89-56   T 33   ☑ 33  
Q 750+1   T 751 

...

Q 90-828  T -738 ☑ -738
Q 46-234  T -188 ☒ -189
Q 837-7   T 830  ☑ 830 
Q 93-61   T 32   ☒ 31  
Q 447-221 T 226  ☒ 137 
Q 78+69   T 147  ☑ 147 
Q 65+17   T 82   ☑ 82  
Q 583-3   T 580  ☒ 570 
Q 602-20  T 582  ☒ 581 
Q 675+44  T 719  ☑ 719 
Q 968+95  T 1063 ☑ 1063
Q 648+23  T 671  ☒ 672 
Q 464-995 T -531 ☒ -540
Q 52-924  T -872 ☒ -860
Q 2+343   T 345  ☑ 345 
Q 496+84  T 580  ☑ 580 
Q 81-21   T 60   ☒ 61  
Q 21-434  T -413 ☑ -413
Q 10-67   T -57  ☑ -57 
Q 6-694   T -688 ☑ -688
Q 38+971  T 1009 ☒ 1000
Q 181-258 T -77  ☒ -37 
Q 218+123 T 341  ☒ 321 
Q 475-832 T -357 ☑ -357
Q 17+993  T 1010 ☒ 1000
Q 995-18  T 977  ☒ 976 
Q 767-350 T 417  ☒ 317 
Q 96+90   T 186  ☒ 176 
Q 60+95   T 155  ☑ 155 
Q 58+895  T 953  ☑ 953 
Q 51-844  T -793 ☒ -893
Q 721+45  T 766  ☑ 766 
Q 25+985  T 1010 ☒ 1000
Q 564+751 T 1315 ☒ 1306
Q 340+41  T 381  ☑ 381 
Q 13+653  T 666  ☒ 667 
Q 712+284 T 996  ☒ 1087
Q 32+25   T 57   ☑ 57  
Q 456+247 T 703  ☒ 603 
Q 13+78   T 91   ☑ 91  
Q 259-147 T 112  ☒ 11  
Q 95+944  T 1039

Q 0+878   T 878  ☑ 878 
Q 28-120  T -92  ☑ -92 
Q 764-816 T -52  ☒ -56 
Q 328-21  T 307  ☑ 307 
Q 5+818   T 823  ☑ 823 
Q 448+675 T 1123 ☒ 1133
Q 67+523  T 590  ☑ 590 
Q 42-708  T -666 ☑ -666
Q 0-910   T -910 ☒ -911
Q 915-42  T 873  ☑ 873 
Q 386-80  T 306  ☑ 306 
Q 213+24  T 237  ☒ 248 
Q 0-510   T -510 ☑ -510
Q 180-1   T 179  ☑ 179 
Q 81-763  T -682 ☑ -682
Q 893-43  T 850  ☑ 850 
Q 882-90  T 792  ☒ 892 
Q 475-327 T 148  ☒ 337 
Q 92+412  T 504  ☑ 504 
Q 9+581   T 590  ☒ 580 
Q 270+8   T 278  ☑ 278 
Q 24-548  T -524 ☒ -533
Q 46-654  T -608 ☑ -608
Q 991-8   T 983  ☒ 973 
Q 389-400 T -11  ☒ -19 
Q 547-50  T 497  ☒ 597 
Q 174+57  T 231  ☑ 231 
Q 36-805  T -769 ☒ -779
Q 92-635  T -543 ☒ -553
Q 169-750 T -581 ☒ -582
Q 828+65  T 893  ☑ 893 
Q 721+794 T 1515 ☒ 1505
Q 814+81  T 895  ☑ 895 
Q 450-433 T 17   ☒ 29  
Q 0+962   T 962  ☑ 962 
Q 205+9   T 214  ☑ 214 
Q 443-62  T 381  ☑ 381 
Q 651+85  T 736  ☑ 736 
Q 914-94  T 820  ☒ 810 
Q 0-219   T -219 ☒ -210
Q 5-808   T -803 ☑ -803
Q 3+791   T 794 

Q 25-984  T -959 ☒ -960
Q 53+96   T 149  ☑ 149 
Q 32+782  T 814  ☑ 814 
Q 59+46   T 105  ☑ 105 
Q 820-820 T 0    ☒ -99 
Q 540-504 T 36   ☒ 35  
Q 964+597 T 1561 ☒ 1461
Q 38+330  T 368  ☑ 368 
Q 514-64  T 450  ☑ 450 
Q 919-190 T 729  ☒ 739 
Q 950+9   T 959  ☑ 959 
Q 860-7   T 853  ☑ 853 
Q 2-561   T -559 ☒ -569
Q 559-695 T -136 ☒ -155
Q 758-543 T 215  ☒ 206 
Q 274+0   T 274  ☑ 274 
Q 95+504  T 599  ☒ 609 
Q 79-446  T -367 ☑ -367
Q 7+863   T 870  ☑ 870 
Q 922+7   T 929  ☒ 928 
Q 400-14  T 386  ☑ 386 
Q 306+6   T 312  ☑ 312 
Q 0-905   T -905 ☑ -905
Q 6-823   T -817 ☑ -817
Q 4+531   T 535  ☑ 535 
Q 69+685  T 754  ☒ 744 
Q 296-2   T 294  ☑ 294 
Q 282-0   T 282  ☑ 282 
Q 427+70  T 497  ☑ 497 
Q 10-73   T -63  ☒ -62 
Q 676+6   T 682  ☑ 682 
Q 55+614  T 669  ☑ 669 
Q 339+44  T 383  ☑ 383 
Q 47+204  T 251  ☒ 252 
Q 506+691 T 1197 ☒ 1108
Q 4-803   T -799 ☑ -799
Q 14-67   T -53  ☒ -52 
Q 7+802   T 809  ☒ 808 
Q 61+703  T 764  ☒ 774 
Q 12-213  T -201 ☒ -209
Q 28+523  T 551  ☒ 561 
Q 688+8   T 696 

Q 353+24  T 377  ☑ 377 
Q 494+363 T 857  ☒ 758 
Q 680-206 T 474  ☒ 485 
Q 657-0   T 657  ☑ 657 
Q 42-154  T -112 ☒ -119
Q 152-94  T 58   ☒ 51  
Q 753+472 T 1225 ☒ 1215
Q 6+488   T 494  ☑ 494 
Q 44+718  T 762  ☑ 762 
Q 1+494   T 495  ☒ 494 
Q 295-333 T -38  ☒ -33 
Q 737-974 T -237 ☒ -247
Q 141+960 T 1101 ☒ 1100
Q 506-142 T 364  ☒ 363 
Q 593+43  T 636  ☑ 636 
Q 60+576  T 636  ☑ 636 
Q 484+474 T 958  ☒ 968 
Q 345-4   T 341  ☑ 341 
Q 710-32  T 678  ☒ 688 
Q 6+715   T 721  ☑ 721 
Q 7-174   T -167 ☑ -167
Q 662+212 T 874  ☒ 884 
Q 13+94   T 107  ☒ 106 
Q 497+51  T 548  ☑ 548 
Q 916+888 T 1804 ☑ 1804
Q 7+205   T 212  ☑ 212 
Q 291+66  T 357  ☑ 357 
Q 74-66   T 8    ☒ 9   
Q 6-562   T -556 ☑ -556
Q 144+679 T 823  ☒ 813 
Q 836+103 T 939  ☑ 939 
Q 34+463  T 497  ☑ 497 
Q 116-891 T -775 ☒ -710
Q 734+5   T 739  ☒ 749 
Q 45-804  T -759 ☒ -769
Q 1-742   T -741 ☑ -741
Q 164-84  T 80   ☒ 79  
Q 50+966  T 1016 ☒ 1006
Q 437+58  T 495  ☑ 495 
Q 41-769  T -728 ☑ -728
Q 622+27  T 649  ☒ 648 
Q 825-65  T 760 

Q 87+858  T 945  ☑ 945 
Q 13-761  T -748 ☒ -747
Q 167-422 T -255 ☒ -247
Q 46-22   T 24   ☒ 26  
Q 431-2   T 429  ☒ 439 
Q 211-326 T -115 ☒ -11 
Q 36-351  T -315 ☑ -315
Q 70+661  T 731  ☑ 731 
Q 89-38   T 51   ☒ 59  
Q 152-19  T 133  ☒ 134 
Q 561-5   T 556  ☑ 556 
Q 3-948   T -945 ☑ -945
Q 63+6    T 69   ☑ 69  
Q 1+898   T 899  ☑ 899 
Q 890+159 T 1049 ☒ 1078
Q 806-185 T 621  ☒ 618 
Q 8-568   T -560 ☑ -560
Q 90+495  T 585  ☑ 585 
Q 689-63  T 626  ☑ 626 
Q 6-800   T -794 ☑ -794
Q 929+536 T 1465 ☒ 1455
Q 43+437  T 480  ☑ 480 
Q 979+24  T 1003 ☒ 1013
Q 948+888 T 1836 ☒ 1887
Q 88+654  T 742  ☒ 741 
Q 37-380  T -343 ☑ -343
Q 372+667 T 1039 ☒ 103 
Q 32-296  T -264 ☑ -264
Q 59-31   T 28   ☒ 27  
Q 7+147   T 154  ☑ 154 
Q 951-582 T 369  ☒ 383 
Q 354+1   T 355  ☑ 355 
Q 8-833   T -825 ☒ -834
Q 40+885  T 925  ☒ 915 
Q 13-692  T -679 ☒ -689
Q 478-52  T 426  ☑ 426 
Q 75+479  T 554  ☑ 554 
Q 287+9   T 296  ☑ 296 
Q 110+20  T 130  ☑ 130 
Q 90-307  T -217 ☑ -217
Q 617+4   T 621  ☑ 621 
Q 980+29  T 1009